<h1>Segmentation_Clustering_Test

<h2>First Part

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [2]:
#Get the right url
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [3]:
data  = requests.get(url).text

In [4]:
soup = BeautifulSoup(data,"html5lib")

In [5]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

In [6]:
#By scraping the wikipedia page with Beautiful Soup and selection of content with "if row.span.text" the 'Not assigned' 
    #postal codes are not considered for the table_contents
#if you run the code: "print(table_contents)" you see the table_content

In [7]:
df_Toronto =pd.DataFrame(table_contents)
df_Toronto['Borough']=df_Toronto['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [8]:
df_Toronto.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [9]:
df_Toronto.shape

(103, 3)

<h2>Second Part

In [10]:
#geocoder package was not working
#the csv file must be used: 

df_coordinates = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv')

In [11]:
df_coordinates.index

RangeIndex(start=0, stop=103, step=1)

In [12]:
df_coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [13]:
#Rename column for merge with df_Toronto
df_coordinates = df_coordinates.rename(columns={"Postal Code": "PostalCode"})
df_coordinates.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
#merge dataframes for new dataframe = result
Toronto_geo = pd.merge(df_Toronto, df_coordinates, how="outer", on="PostalCode")
Toronto_geo

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto Business,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


<h2>Third Part

<h3>Explore

In [15]:
#Boroughs that contain the word Toronto

Neighbour = Toronto_geo["Borough"].str.contains("Toronto")
Toronto_geo.loc[Neighbour, :].head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [16]:
#pick on borough for to explore and cluster the neighborhood 

neighborhood_latitude = Toronto_geo.loc[2, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Toronto_geo.loc[2, 'Longitude'] # neighborhood longitude value

In [17]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '3LQFHDAYOBHTTF5F42RVQIQ34AHSBRH12MJKLOFJSGL5AI01' # your Foursquare Secret
ACCESS_TOKEN = 'MMJ4I0VUSS1VG0WWZVCFUOL4PGSUTEURLVQF5OZSCSSKB2EO' # your FourSquare Access Token
VERSION = '20180605' # Foursquare API version

radius = 500 # Radius to explore
LIMIT = 100 # A default Foursquare API limit value


In [18]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&radius={}&limit={}'.format(
    CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, ACCESS_TOKEN, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=&client_secret=3LQFHDAYOBHTTF5F42RVQIQ34AHSBRH12MJKLOFJSGL5AI01&ll=43.6542599,-79.3606359&oauth_token=MMJ4I0VUSS1VG0WWZVCFUOL4PGSUTEURLVQF5OZSCSSKB2EO&v=20180605&radius=500&limit=100'

In [30]:
# Get the data
results = requests.get(url).json()

In [20]:
# Work with the data

from pandas.io.json import json_normalize

venues = results['response']['venues']

dataframe = json_normalize(venues)
dataframe.head()


<ipython-input-20-cac2ff7a4ca6>:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  dataframe = json_normalize(venues)


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,location.postalCode,venuePage.id
0,4bc70f5d14d7952126a066e9,Sackville Playground,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",v-1619390988,False,420 king st E,43.654656,-79.359871,"[{'label': 'display', 'lat': 43.65465604258614...",75,CA,Toronto,ON,Canada,"[420 king st E, Toronto ON, Canada]",NaN,NaN,NaN
1,4dc9d4d9d16495ca5add0803,Cam's Auto Service,"[{'id': '4bf58dd8d48988d124951735', 'name': 'A...",v-1619390988,False,475 King Street East,43.654195,-79.360545,"[{'label': 'display', 'lat': 43.65419500779484...",10,CA,Toronto,ON,Canada,"[475 King Street East (Sackville Street), Toro...",Sackville Street,M4M 2T7,NaN
2,5e5d749285a0610007e60fe8,Terroni Sud Forno Produzione e Spaccio,"[{'id': '4bf58dd8d48988d1f5941735', 'name': 'G...",v-1619390988,False,22 Sackville St,43.653903,-79.360018,"[{'label': 'display', 'lat': 43.653903, 'lng':...",63,CA,Toronto,ON,Canada,"[22 Sackville St, Toronto ON M5A 3E2, Canada]",NaN,M5A 3E2,NaN
3,53b8466a498e83df908c3f21,Tandem Coffee,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1619390988,False,368 King St E,43.653559,-79.361809,"[{'label': 'display', 'lat': 43.65355870959944...",122,CA,Toronto,ON,Canada,"[368 King St E (at Trinity St), Toronto ON, Ca...",at Trinity St,NaN,NaN
4,5bdc6c2bba57b4002c4c71a8,Oldtown Bodega,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1619390988,False,NaN,43.653966,-79.360752,"[{'label': 'display', 'lat': 43.653966, 'lng':...",34,CA,Toronto,ON,Canada,"[Toronto ON M5A 1L6, Canada]",NaN,M5A 1L6,NaN


In [33]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered


,name,categories,address,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,crossStreet,postalCode,id
0,Sackville Playground,Park,420 king st E,43.654656,-79.359871,"[{'label': 'display', 'lat': 43.65465604258614...",75,CA,Toronto,ON,Canada,"[420 king st E, Toronto ON, Canada]",NaN,NaN,4bc70f5d14d7952126a066e9
1,Cam's Auto Service,Automotive Shop,475 King Street East,43.654195,-79.360545,"[{'label': 'display', 'lat': 43.65419500779484...",10,CA,Toronto,ON,Canada,"[475 King Street East (Sackville Street), Toro...",Sackville Street,M4M 2T7,4dc9d4d9d16495ca5add0803
2,Terroni Sud Forno Produzione e Spaccio,Gourmet Shop,22 Sackville St,43.653903,-79.360018,"[{'label': 'display', 'lat': 43.653903, 'lng':...",63,CA,Toronto,ON,Canada,"[22 Sackville St, Toronto ON M5A 3E2, Canada]",NaN,M5A 3E2,5e5d749285a0610007e60fe8
3,Tandem Coffee,Coffee Shop,368 King St E,43.653559,-79.361809,"[{'label': 'display', 'lat': 43.65355870959944...",122,CA,Toronto,ON,Canada,"[368 King St E (at Trinity St), Toronto ON, Ca...",at Trinity St,NaN,53b8466a498e83df908c3f21
4,Oldtown Bodega,Café,NaN,43.653966,-79.360752,"[{'label': 'display', 'lat': 43.653966, 'lng':...",34,CA,Toronto,ON,Canada,"[Toronto ON M5A 1L6, Canada]",NaN,M5A 1L6,5bdc6c2bba57b4002c4c71a8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Ward 28 Voting Station,City Hall,570 Queen E,43.654830,-79.357985,"[{'label': 'display', 'lat': 43.65483, 'lng': ...",222,CA,Toronto,ON,Canada,"[570 Queen E, Toronto ON, Canada]",NaN,NaN,4cc5a24d82388cfa1ec27635
96,STACKLAB,Design Studio,28 Eastern Avenue,43.653334,-79.359838,"[{'label': 'display', 'lat': 43.65333396303207...",121,CA,Toronto,ON,Canada,"[28 Eastern Avenue (Parliament Street), Toront...",Parliament Street,M5A 1H5,552ff7b5498e086325bc13ae
97,Bright Street Playground,Park,50 Bright Street,43.656105,-79.359741,"[{'label': 'display', 'lat': 43.65610515821674...",217,CA,Toronto,ON,Canada,"[50 Bright Street (King), Toronto ON, Canada]",King,NaN,4ce2d03af8a4a1439806f3bc
98,Corktown Cottages,Bed & Breakfast,2A Sumach St,43.654332,-79.358005,"[{'label': 'display', 'lat': 43.6543323006481,...",212,CA,Toronto,ON,Canada,"[2A Sumach St, Toronto ON M5A 3J4, Canada]",NaN,M5A 3J4,56550a3d498edc3a6b1b7e63


In [37]:
dataframe_filtered['categories'].str.contains("Park")


#Neighbour = Toronto_geo["Borough"].str.contains("Toronto")

0      True
1     False
2     False
3     False
4     False
      ...  
95    False
96    False
97     True
98    False
99    False
Name: categories, Length: 100, dtype: object

<h3>Clustering

In [23]:
# Clustering on all Borough of Toronto

Toronto_geo_grouped = Toronto_geo.groupby('Borough').mean().reset_index()
Toronto_geo_grouped.head()

,Borough,Latitude,Longitude
0,Central Toronto,43.701980,-79.398954
1,Downtown Toronto,43.654624,-79.384184
2,Downtown Toronto Stn A,43.646435,-79.374846
3,East Toronto,43.671110,-79.325428
4,East Toronto Business,43.662744,-79.321558


In [24]:
# import kmeans from clustering stage
from sklearn.cluster import KMeans

#set number of clusters
kclusters = 4

Toronto_geo_grouped_clustering = Toronto_geo_grouped.drop('Borough', 1)

#run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_geo_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 2, 2, 2, 2, 2, 3, 3, 3])

In [25]:
Toronto_geo_grouped.insert(0, 'Cluster Labels', kmeans.labels_)
Toronto_geo_grouped.head()

,Cluster Labels,Borough,Latitude,Longitude
0,0,Central Toronto,43.701980,-79.398954
1,2,Downtown Toronto,43.654624,-79.384184
2,2,Downtown Toronto Stn A,43.646435,-79.374846
3,2,East Toronto,43.671110,-79.325428
4,2,East Toronto Business,43.662744,-79.321558


In [26]:
Toronto_geo_grouped.loc[Toronto_geo_grouped['Cluster Labels'] == 0, Toronto_geo_grouped.columns[[1] + list(range(5, Toronto_geo_grouped.shape[1]))]]

,Borough
0,Central Toronto
10,North York
13,West Toronto
14,York


In [27]:
Toronto_geo_grouped.loc[Toronto_geo_grouped['Cluster Labels'] == 1, Toronto_geo_grouped.columns[[1] + list(range(5, Toronto_geo_grouped.shape[1]))]]

,Borough
12,Scarborough


In [28]:
Toronto_geo_grouped.loc[Toronto_geo_grouped['Cluster Labels'] == 2, Toronto_geo_grouped.columns[[1] + list(range(5, Toronto_geo_grouped.shape[1]))]]

,Borough
1,Downtown Toronto
2,Downtown Toronto Stn A
3,East Toronto
4,East Toronto Business
5,East York
6,East York/East Toronto
11,Queen's Park


In [29]:
Toronto_geo_grouped.loc[Toronto_geo_grouped['Cluster Labels'] == 3, Toronto_geo_grouped.columns[[1] + list(range(5, Toronto_geo_grouped.shape[1]))]]

,Borough
7,Etobicoke
8,Etobicoke Northwest
9,Mississauga
